In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salaries-of-us-presidents-and-vice-presidents/pvp_dataset.csv


In [2]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 97.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 37.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:000:00:0100

In [3]:
import kagglehub

#download salaries dataset from kaggle
path = kagglehub.dataset_download("adiquant/salaries-of-us-presidents-and-vice-presidents")

print("Path to dataset files:", path)


Path to dataset files: /kaggle/input/salaries-of-us-presidents-and-vice-presidents


In [5]:
import pandas as pd
import os

#Load the dataset 
df = pd.read_csv("/kaggle/input/salaries-of-us-presidents-and-vice-presidents/pvp_dataset.csv")


   year               name         party    term  salary  \
0  1789  Washington,George  Unaffiliated   First   25000   
1  1790  Washington,George  Unaffiliated   First   25000   
2  1791  Washington,George  Unaffiliated   First   25000   
3  1792  Washington,George  Unaffiliated   First   25000   
4  1793  Washington,George  Unaffiliated  Second   25000   

                   position_title  
0  PRESIDENT OF THE UNITED STATES  
1  PRESIDENT OF THE UNITED STATES  
2  PRESIDENT OF THE UNITED STATES  
3  PRESIDENT OF THE UNITED STATES  
4  PRESIDENT OF THE UNITED STATES  


In [6]:
df.columns = ["Year", "President", "party", "term","PresidentSalary", "title"]


In [7]:

qa_data = {
    "Instruction": [],
    "Input": [],
    "Response": []
}
#adds conversation text to the dataset because it is mostly single word answers
for _, row in df.iterrows():
    qa_data["Instruction"].append("What was the salary of the U.S. president in " + str(row["Year"]) + "?")
    qa_data["Input"].append("")
    qa_data["Response"].append(f"In {row['Year']}, the U.S. president earned ${row['PresidentSalary']}.")

    qa_data["Instruction"].append("Who was the U.S. president in " + str(row["Year"]) + "?")
    qa_data["Input"].append("")
    qa_data["Response"].append(f"The U.S. president in {row['Year']} was {row['President']}.")

qa_df = pd.DataFrame(qa_data)

qa_df.to_csv("fine_tune_presidents.csv", index=False)
print("Formatted dataset saved!")


Formatted dataset saved!


In [8]:
from datasets import Dataset

df = pd.read_csv("fine_tune_presidents.csv")
dataset = Dataset.from_pandas(df)
#prompt formating
def formatting_prompts_func(examples):
    texts = []
    for inst, inp, resp in zip(examples["Instruction"], examples["Input"], examples["Response"]):
        prompt = f"""### Instruction:
{inst}

### Input:
{inp}

### Response:
{resp}"""
        texts.append(prompt)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/858 [00:00<?, ? examples/s]

In [ ]:



from unsloth import FastLanguageModel
from trl import SFTTrainer, SFTConfig
import torch
import wandb
#Load Llama 
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True
)

#lora
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    random_state=3407,
    bias="none",
    use_gradient_checkpointing="unsloth",
    use_rslora=False,
    loftq_config=None,
)

#training param
training_args = SFTConfig(
    output_dir="./fine_tuned_model",
    max_seq_length=2048,
    max_steps=60, 
    learning_rate=2e-4
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=1,
    args=training_args,
)
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_DISABLED"] = "true"
wandb.init(mode="disabled")

#train
trainer_stats = trainer.train()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2025.3.14 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


Unsloth: Tokenizing ["text"]:   0%|          | 0/858 [00:00<?, ? examples/s]

In [ ]:
FastLanguageModel.for_inference(model)

question = "What was George Washingtons salary in his first term?"
inputs = tokenizer([f"### Instruction:\n{question}\n\n### Response:\n"], return_tensors="pt").to("cuda")
#generate response
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0].split("### Response:")[1])
